In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AffinityPropagation
from collections import Counter
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram

import cPickle as pickle
import graphlab

In [2]:
pd.options.display.max_colwidth = 100

In [12]:
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# from textblob.sentiments import NaiveBayesAnalyzer
import re
import pandas as pd
import sqlite3
# from textblob import TextBlob

import seaborn as sns
sns.set(color_codes=True)

# encoding=utf8
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

Create Review Recommender.ipynb  Use Review Recommender.ipynb


In [9]:
df = pd.read_csv('../data/pitchfork_no_nlp.csv')

years = np.arange(1999, 2018)
df_new = df[df['new_album'] == 1]
df_reissue = df[df['new_album'] == 0]

df_spotify = pd.read_csv('../data/spotify_with_rank.csv')
df_spotify = df_spotify.groupby(['title', 'artist', 'album_uri']).mean().reset_index()

df_merged = df.merge(df_spotify, how='left', on=['artist', 'title'])
df_merged = df_merged.drop_duplicates(['title', 'artist', 'reviewid'], keep='first')


In [13]:
# 4. Limit the number of features and see if the words of the topics change.
vectorizer = TfidfVectorizer(stop_words='english', max_features=500, max_df=.4)
X = vectorizer.fit_transform(df_merged['content'].values.astype('U'))

In [14]:
df_merged['tf-idf'] = list(X.toarray())

df_merged['danceability rounded'] = (df_merged['danceability mean']*10).round(1)
df_merged['valence rounded'] = (df_merged['valence mean']*10).round(1)
df_merged['energy rounded'] = (df_merged['energy mean']*10).round(1)
df_merged['acousticness rounded'] = (df_merged['acousticness mean']*10).round(1)
df_merged['score_bin_5'] = pd.cut(df_merged['score'], bins=5, labels=False)
df_merged['score_bin_2'] = pd.cut(df_merged['score'], bins=2, labels=False)
df_merged['danceability rounded'].fillna(value=df_merged['danceability rounded'].mean(), inplace=True)
df_merged['valence rounded'].fillna(value=df_merged['valence rounded'].mean(), inplace=True)
df_merged['energy rounded'].fillna(value=df_merged['energy rounded'].mean(), inplace=True)
df_merged['acousticness rounded'].fillna(value=df_merged['acousticness rounded'].mean(), inplace=True)
df_merged['spotify_match'] = np.where(df_merged['year sum'] > 0, 1, 0)

df_merged.drop_duplicates(['title', 'artist'], keep='first')
df_merged =df_merged.dropna(subset=['content'])

In [83]:
sf = graphlab.SFrame(df_merged[['url', 'tf-idf', 
                                'danceability rounded', 'valence rounded', 'energy rounded', 'acousticness rounded', 
                                'pub_year', u'genre_electronic', u'genre_experimental',
       u'genre_folk/country', u'genre_global', u'genre_jazz', u'genre_metal',
       u'genre_pop/r&b', u'genre_rap', u'genre_rock', 'new_album', 'spotify_match']])

sf2 = graphlab.SFrame(df_merged[['url', 'tf-idf', 'pub_year', u'genre_electronic', u'genre_experimental',
       u'genre_folk/country', u'genre_global', u'genre_jazz', u'genre_metal',
       u'genre_pop/r&b', u'genre_rap', u'genre_rock']])

In [84]:
recommender = graphlab.recommender.item_content_recommender.create(sf, item_id='url')

('Applying transform:\n', Class             : AutoVectorizer

Model Fields
------------
Features          : ['tf-idf', 'danceability rounded', 'valence rounded', 'energy rounded', 'acousticness rounded', 'pub_year', 'genre_electronic', 'genre_experimental', 'genre_folk/country', 'genre_global', 'genre_jazz', 'genre_metal', 'genre_pop/r&b', 'genre_rap', 'genre_rock', 'new_album', 'spotify_match']
Excluded Features : ['url']

Column                Type   Interpretation  Transforms   Output Type
--------------------  -----  --------------  -----------  -----------
tf-idf                array  vector          None         array      
danceability rounded  float  numerical       None         float      
valence rounded       float  numerical       None         float      
energy rounded        float  numerical       None         float      
acousticness rounded  float  numerical       None         float      
pub_year              int    numerical       None         int        
genre_electr

Recsys training: model = item_content_recommender

Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

Starting pairwise querying.

...

Data has 0 observations with 0 users and 19619 items.

Data prepared in: 1.93364s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.107563s

In [108]:
recommender_tfidf = graphlab.recommender.item_content_recommender.create(sf2, item_id='url')

('Applying transform:\n', Class             : AutoVectorizer

Model Fields
------------
Features          : ['tf-idf', 'pub_year', 'genre_electronic', 'genre_experimental', 'genre_folk/country', 'genre_global', 'genre_jazz', 'genre_metal', 'genre_pop/r&b', 'genre_rap', 'genre_rock']
Excluded Features : ['url']

Column              Type   Interpretation  Transforms   Output Type
------------------  -----  --------------  -----------  -----------
tf-idf              array  vector          None         array      
pub_year            int    numerical       None         int        
genre_electronic    int    categorical     astype(str)  str        
genre_experimental  int    categorical     astype(str)  str        
genre_folk/country  int    categorical     astype(str)  str        
genre_global        int    categorical     astype(str)  str        
genre_jazz          int    categorical     astype(str)  str        
genre_metal         int    categorical     astype(str)  str        
genre_p

Recsys training: model = item_content_recommender

Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

Starting pairwise querying.

...

Data has 0 observations with 0 users and 19619 items.

Data prepared in: 2.37619s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.131057s

In [93]:
#Save recommenders and DataFrame

recommender.save('../recommender')

recommender_tfidf.save('../recommender_tfidf')

df_merged.to_csv('../recommender_df.csv')

In [85]:
def print_top_url(df_similar):
    df_copy = df_similar.reset_index()
    for i in range(len(df_similar['url'])):
        print "{}, {}".format(df_copy['artist'][i].title(), df_copy['title'][i].title())
        print "http://open.spotify.com/album/{}".format(df_copy['album_uri'][i])
        print df_copy['url'][i]
        print "\n"

In [86]:
def get_similar(df, url, recommender):
    top_5_similar = recommender.get_similar_items([url], k=20)['similar']
    return df[(df['score'] > 7) & 
               (df['url'].isin(top_5_similar))][['artist','title', 'score', 'url', 'danceability rounded',
                                                'valence rounded', 'energy rounded', 'acousticness rounded', 
                                               'pub_year', 'spotify_match', 'album_uri']].head(10)
    

In [87]:
def get_url(artist, title):
    return df[(df['artist'] == artist) & (df['title'] == title)].iloc[0][6]

In [88]:
len(df_merged[df_merged['artist'] == 'jlkjk']['title'])

0

In [111]:
artist = raw_input("Artist: ").strip().lower()
while True:
    if len(df_merged[df_merged['artist'] == artist]['title']) == 0:
        print "Try another artist"  
    else:
        break
    artist = raw_input("Artist: ").strip().lower()
print '\n'
print 'Select Album From Below:'
for each in df_merged[df_merged['artist'] == artist]['title']:
            print each
print '\n'
title = raw_input("Album: ").strip().lower()
while True:
    try:
        url = get_url(artist, title)
        break
    except:
        print "Not a valid album. Try again from what's listed above"
        title = raw_input("Album: ").lower()
        url = get_url(artist, title)

df_similar = get_similar(df_merged, url, recommender_tfidf)
print '\n'
print_top_url(df_similar)

Artist: joanna newsom


Select Album From Below:
divers
have one on me
joanna newsom & the ys street band ep
ys
the milk-eyed mender


Album: divers


Larkin Grimm, Chasing An Illusion
http://open.spotify.com/album/1dU8ajuDtfyZogbFgsT3EV
https://pitchfork.com/reviews/albums/larkin-grimm-chasing-an-illusion/


Hiss Golden Messenger, Heart Like A Levee
http://open.spotify.com/album/4rEEkztpEU1GBmy5uBYHUS
https://pitchfork.com/reviews/albums/22323-heart-like-a-levee/


Lee Hazlewood, The Very Special World Of Lee Hazlewood
http://open.spotify.com/album/4BshrFXTkg7Vhtdcqg0FpX
https://pitchfork.com/reviews/albums/21461-the-very-special-world-of-lee-hazlewood-lee-hazlewoodism-its-cause-and-cure-something-special/


Nadia Reid, Listen To Formation, Look For The Signs
http://open.spotify.com/album/5ZDAvkRKsM1s1O9BVqRWYt
https://pitchfork.com/reviews/albums/21297-listen-to-formation-look-for-the-signs/


Monk Parker, How The Spark Loves The Tinder
http://open.spotify.com/album/nan
https://pitch

In [92]:
type(recommender)

graphlab.toolkits.recommender.item_content_recommender.ItemContentRecommender

In [505]:
url = get_url('kanye west', 'yeezus')

In [506]:
url

'https://pitchfork.com/reviews/albums/18172-kanye-west-yeezus/'

In [41]:
get_similar(df_merged, url, recommender)

,artist,title,score,url,danceability rounded,valence rounded,energy rounded,acousticness rounded,pub_year,spotify_match,album_uri
2811,deerhunter,fading frontier,8.4,https://pitchfork.com/reviews/albums/21039-fading-frontier/,5.0,4.7,6.8,1.2,2015,1,0y5XgOUEiJSSIMxC4ALRAC
3119,tame impala,currents,9.3,https://pitchfork.com/reviews/albums/20578-currents/,5.0,4.7,7.1,1.6,2015,1,79dL7FLiJFOO0EoehUHQBv
3725,the sidekicks,runners in the nerved world,7.4,https://pitchfork.com/reviews/albums/20139-runners-in-the-nerved-world/,5.1,5.2,7.4,1.3,2015,1,2lzS0Mb1A7ndVfiY5TgTmV
4486,the pains of being pure at heart,days of abandon,7.5,https://pitchfork.com/reviews/albums/19335-the-pains-of-being-pure-at-heart-days-of-abandon/,5.0,4.9,6.5,1.4,2014,1,2lV0RUPATtdoDFxRtV605S
5128,arcade fire,reflektor,9.2,https://pitchfork.com/reviews/albums/18667-arcade-fire-reflektor/,4.8,4.8,6.6,1.6,2013,1,3Y2bjF2Jwocc0USbu08Y2I
5129,arcade fire,reflektor,9.2,https://pitchfork.com/reviews/albums/18667-arcade-fire-reflektor/,4.8,4.7,6.6,1.7,2013,1,4E0m7AIVc2d2QZMrMNXdMZ
5169,paul mccartney,new,7.8,https://pitchfork.com/reviews/albums/18634-paul-mccartney-new/,5.5,4.3,6.9,1.3,2013,1,216q1rvAofGBh1CPyl0Qzl
5349,disappears,era,7.3,https://pitchfork.com/reviews/albums/18430-disappears-era/,5.3,3.9,6.8,1.0,2013,1,2GvxR9cvRQK6SzJ6fFfTY9
6050,foals,holy fire,7.6,https://pitchfork.com/reviews/albums/17578-holy-fire/,5.5,4.3,6.9,1.4,2013,1,0PIR7PK8DMB4pgoxq7F9Ad
9364,broken social scene,forgiveness rock record,8.3,https://pitchfork.com/reviews/albums/14200-forgiveness-rock-record/,5.0,4.1,6.3,0.9,2010,1,2LHfGLiEaV595PIYmCPMmh


In [399]:
get_similar(df_merged, 'https://pitchfork.com/reviews/albums/17253-good-kid-maad-city/', recommender)

,artist,title,score,url,danceability rounded,valence rounded,energy rounded,acousticness rounded,pub_year,spotify_match
2708,g herbo,ballin like im kobe,7.2,https://pitchfork.com/reviews/albums/21153-ballin-like-im-kobe/,5.8,3.5,6.6,1.2,2015,1
3070,meek mill,dreams worth more than money,7.4,https://pitchfork.com/reviews/albums/20778-dreams-worth-more-than-money/,6.3,3.5,6.5,1.4,2015,1
3191,a$ap rocky,at.long.last.a$ap,7.8,https://pitchfork.com/reviews/albums/20661-atlonglastaap/,6.0,2.9,6.0,1.9,2015,1
3837,run the jewels,run the jewels 2,9.0,https://pitchfork.com/reviews/albums/19968-run-the-jewels-run-the-jewels-2/,6.2,3.3,6.4,1.0,2014,1
3962,"freeway, the jacka",highway robbery,7.9,https://pitchfork.com/reviews/albums/19743-freeway-the-jacka-highway-robbery/,6.0,4.0,6.5,0.7,2014,1
5101,pusha t,my name is my name,8.0,https://pitchfork.com/reviews/albums/18277-pusha-t-my-name-is-my-name/,5.8,4.3,6.9,1.1,2013,1
5164,drake,nothing was the same,8.6,https://pitchfork.com/reviews/albums/18511-drake-nothing-was-the-same/,6.6,3.3,5.7,1.5,2013,1
5497,kanye west,yeezus,9.5,https://pitchfork.com/reviews/albums/18172-kanye-west-yeezus/,5.6,3.5,5.9,1.7,2013,1
6871,future,pluto,7.8,https://pitchfork.com/reviews/albums/16554-pluto/,6.3,4.0,6.5,0.8,2012,1
7398,nacho picasso,for the glory,7.9,https://pitchfork.com/reviews/albums/15976-for-the-glory/,6.6,3.3,5.9,1.6,2011,1


In [407]:
get_similar(df_merged, 'https://pitchfork.com/reviews/albums/3160-welcome-interstate-managers/', recommender)

,artist,title,score,url,danceability rounded,valence rounded,energy rounded,acousticness rounded,pub_year,spotify_match
13347,brakes,the beatific visions,7.8,https://pitchfork.com/reviews/albums/9667-the-beatific-visions/,4.7,5.8,7.2,1.8,2006,1
15396,the national,alligator,7.9,https://pitchfork.com/reviews/albums/5719-alligator/,5.2,5.4,7.4,2.8,2005,1
16809,oranger,shutdown the sun,7.3,https://pitchfork.com/reviews/albums/6022-shutdown-the-sun/,5.3,5.8,6.8,1.2,2004,1
17105,david bowie,reality,7.3,https://pitchfork.com/reviews/albums/880-reality/,5.7,5.0,7.3,1.7,2003,1
17226,the natural history,beat beat heartbeat,7.8,https://pitchfork.com/reviews/albums/5877-beat-beat-heartbeat/,5.9,6.5,7.0,2.2,2003,1
17417,black lipstick,converted thieves,7.5,https://pitchfork.com/reviews/albums/769-converted-thieves/,5.0,6.5,7.1,1.2,2003,1
18294,liars,they threw us all in a trench and stuck a monument on top,8.1,https://pitchfork.com/reviews/albums/4764-they-threw-us-all-in-a-trench-and-stuck-a-monument-on-...,5.4,5.6,6.8,1.5,2002,1
18318,skeleton key,obtainium,7.3,https://pitchfork.com/reviews/albums/7220-obtainium/,5.7,5.5,7.4,2.3,2002,1


In [414]:
get_similar(df_merged, 'https://pitchfork.com/reviews/albums/23070-whiteout-conditions/', recommender)

,artist,title,score,url,danceability rounded,valence rounded,energy rounded,acousticness rounded,pub_year,spotify_match
905,white reaper,the worlds best american band,8.0,https://pitchfork.com/reviews/albums/23064-the-worlds-best-american-band/,5.3,5.0,9.2,1.1,2017,1
1508,joyce manor,cody,7.7,https://pitchfork.com/reviews/albums/22477-cody/,5.2,5.2,8.4,1.0,2016,1
1831,the julie ruin,hit reset,8.2,https://pitchfork.com/reviews/albums/22073-hit-reset/,5.5,4.9,7.9,0.8,2016,1
2100,operators,blue wave,7.5,https://pitchfork.com/reviews/albums/21851-blue-wave/,5.2,5.3,7.9,0.1,2016,1
2435,diiv,is the is are,8.1,https://pitchfork.com/reviews/albums/21462-is-the-is-are/,5.0,4.8,8.3,0.5,2016,1
2765,eagles of death metal,zipper down,7.3,https://pitchfork.com/reviews/albums/21115-zipper-down/,5.2,6.2,8.7,0.7,2015,1
2816,new order,music complete,7.2,https://pitchfork.com/reviews/albums/21036-music-complete/,5.4,5.0,9.0,0.1,2015,1
3480,young guv,ripe 4 luv,7.5,https://pitchfork.com/reviews/albums/20241-ripe-4-luv/,5.4,5.4,8.5,0.6,2015,1
4076,the new pornographers,brill bruisers,7.7,https://pitchfork.com/reviews/albums/19722-the-new-pornographers-brill-bruisers/,5.5,5.7,7.9,1.1,2014,1


In [415]:
recommender

Class                            : ItemContentRecommender

Schema
------
User ID                          : __implicit_user__
Item ID                          : url
Target                           : None
Additional observation features  : 0
User side features               : []
Item side features               : ['url', 'tf-idf', 'danceability rounded', 'valence rounded', 'energy rounded', 'acousticness rounded', 'pub_year', 'genre_electronic', 'genre_experimental', 'genre_folk/country', 'genre_global', 'genre_jazz', 'genre_metal', 'genre_pop/r&b', 'genre_rap', 'genre_rock', 'new_album', 'spotify_match']

Statistics
----------
Number of observations           : 0
Number of users                  : 0
Number of items                  : 19619

Training summary
----------------
Training time                    : 0.0875

Model Parameters
----------------
Model class                      : ItemContentRecommender
threshold                        : 0.0001
similarity_type                  : co